## Merging injury and gamelog data

In [2]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from sklearn.preprocessing import OneHotEncoder

/Users/freedom/anaconda3/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [475]:
# Import files
injury_file = 'injuries.csv'
injury = pd.read_csv(injury_file)

game_file = 'gamelogs.csv'
games = pd.read_csv(game_file)

In [476]:
injury.head()

,Date,DL_length,player_id
0,20000323,60,thomj005
1,20000401,15,alvaw001
2,20000401,15,osuna002
3,20000408,15,hamid001
4,20000408,15,timlm001


In [477]:
print(injury.shape)
injury.drop_duplicates(inplace=True)
print('After dropping duplicates',injury.shape)

(9288, 3)
After dropping duplicates (9043, 3)


In [478]:
# Add a new columns that has the Datetime version of date:
injury['dt_date'] = injury['Date'].apply(lambda x: str(x)[4:6] + '/' + str(x)[6:]+'/' + str(x)[:4])
injury['dt_date'] = pd.to_datetime(injury['dt_date'])

In [479]:
games.head()

,date,num_game,day,v_team,v_league,v_team_game_num,h_team,h_league,h_team_game_num,v_team_score,...,v_pl_9_id,h_pl_1_id,h_pl_2_id,h_pl_3_id,h_pl_4_id,h_pl_5_id,h_pl_6_id,h_pl_7_id,h_pl_8_id,h_pl_9_id
0,20000329,0,Wed,CHN,NL,1,NYN,NL,1,5,...,liebj001,hendr001,hamid001,alfoe001,piazm001,ventr001,belld001,zeilt001,ordor001,hampm001
1,20000330,0,Thu,NYN,NL,2,CHN,NL,2,5,...,reedr002,youne001,bufod001,sosas001,gracm001,rodrh001,andrs001,husoj001,giraj001,farnk001
2,20000403,0,Mon,COL,NL,1,ATL,NL,1,0,...,astap001,veraq001,sandr002,jonec004,jordb001,galaa001,jonea002,peree002,weisw001,maddg002
3,20000403,0,Mon,SFN,NL,1,FLO,NL,1,4,...,hernl003,castl001,gonza002,floyc001,wilsp002,lowem001,millk005,browb003,redmm001,ferna001
4,20000403,0,Mon,LAN,NL,1,MON,NL,1,10,...,browk001,bergp001,vidrj001,whitr001,guerv001,stevl001,barrm003,widgc001,cabro001,hermd001


### Drop duplicate rows:

In [480]:
print(games.shape)
# check duplicates:
games.drop_duplicates(inplace=True)
print('After removing duplicate rows:',games.shape)
games.index = np.arange(0,len(games))

(43720, 96)
After removing duplicate rows: (41292, 96)


In [481]:
games.head()

,date,num_game,day,v_team,v_league,v_team_game_num,h_team,h_league,h_team_game_num,v_team_score,...,v_pl_9_id,h_pl_1_id,h_pl_2_id,h_pl_3_id,h_pl_4_id,h_pl_5_id,h_pl_6_id,h_pl_7_id,h_pl_8_id,h_pl_9_id
0,20000329,0,Wed,CHN,NL,1,NYN,NL,1,5,...,liebj001,hendr001,hamid001,alfoe001,piazm001,ventr001,belld001,zeilt001,ordor001,hampm001
1,20000330,0,Thu,NYN,NL,2,CHN,NL,2,5,...,reedr002,youne001,bufod001,sosas001,gracm001,rodrh001,andrs001,husoj001,giraj001,farnk001
2,20000403,0,Mon,COL,NL,1,ATL,NL,1,0,...,astap001,veraq001,sandr002,jonec004,jordb001,galaa001,jonea002,peree002,weisw001,maddg002
3,20000403,0,Mon,SFN,NL,1,FLO,NL,1,4,...,hernl003,castl001,gonza002,floyc001,wilsp002,lowem001,millk005,browb003,redmm001,ferna001
4,20000403,0,Mon,LAN,NL,1,MON,NL,1,10,...,browk001,bergp001,vidrj001,whitr001,guerv001,stevl001,barrm003,widgc001,cabro001,hermd001


In [482]:
# Combine the v_leage and h_league into one feature: intra_league 
games['intra_league'] = games.apply(lambda x: 1 if x['h_league']==x['v_league'] else 0, axis=1)
games.drop(['v_league','h_league'],axis=1, inplace=True)

# Sort by date, keep the index as a separate column and set index to dt_date

In [485]:
# Make sure the game logs are sorted by date of game and reset the index
games = games.sort_values('date')
games.index = np.arange(0,len(games))

# Add a dt_date column to games table:
games['dt_date'] = games['date'].apply(lambda x: str(x)[4:6] + '/' + str(x)[6:]+'/' + str(x)[:4])
games['dt_date'] = pd.to_datetime(games['dt_date'])

In [486]:
games.head()

,date,num_game,day,v_team,v_team_game_num,h_team,h_team_game_num,v_team_score,h_team_score,game_length_outs,...,h_pl_2_id,h_pl_3_id,h_pl_4_id,h_pl_5_id,h_pl_6_id,h_pl_7_id,h_pl_8_id,h_pl_9_id,intra_league,dt_date
0,20000329,0,Wed,CHN,1,NYN,1,5,3,54,...,hamid001,alfoe001,piazm001,ventr001,belld001,zeilt001,ordor001,hampm001,1,2000-03-29
1,20000330,0,Thu,NYN,2,CHN,2,5,1,66,...,bufod001,sosas001,gracm001,rodrh001,andrs001,husoj001,giraj001,farnk001,1,2000-03-30
2,20000403,0,Mon,CHN,3,SLN,1,1,7,51,...,rente001,lankr001,tatif001,edmoj001,paquc001,davie001,mathm001,kiled001,1,2000-04-03
3,20000403,0,Mon,COL,1,ATL,1,0,2,51,...,sandr002,jonec004,jordb001,galaa001,jonea002,peree002,weisw001,maddg002,1,2000-04-03
4,20000403,0,Mon,SFN,1,FLO,1,4,6,51,...,gonza002,floyc001,wilsp002,lowem001,millk005,browb003,redmm001,ferna001,1,2000-04-03


In [488]:
# I want to keep the current index as a column.
games_ixd = games.reset_index()
games_ixd.rename(columns={"index": "real_index"},inplace=True)
games_ixd.set_index('dt_date',inplace=True)
print(games_ixd.shape)
games_ixd.head()

(41292, 96)


,real_index,date,num_game,day,v_team,v_team_game_num,h_team,h_team_game_num,v_team_score,h_team_score,...,h_pl_1_id,h_pl_2_id,h_pl_3_id,h_pl_4_id,h_pl_5_id,h_pl_6_id,h_pl_7_id,h_pl_8_id,h_pl_9_id,intra_league
dt_date,,,,,,,,,,,,,,,,,,,,,
2000-03-29,0,20000329,0,Wed,CHN,1,NYN,1,5,3,...,hendr001,hamid001,alfoe001,piazm001,ventr001,belld001,zeilt001,ordor001,hampm001,1
2000-03-30,1,20000330,0,Thu,NYN,2,CHN,2,5,1,...,youne001,bufod001,sosas001,gracm001,rodrh001,andrs001,husoj001,giraj001,farnk001,1
2000-04-03,2,20000403,0,Mon,CHN,3,SLN,1,1,7,...,vinaf001,rente001,lankr001,tatif001,edmoj001,paquc001,davie001,mathm001,kiled001,1
2000-04-03,3,20000403,0,Mon,COL,1,ATL,1,0,2,...,veraq001,sandr002,jonec004,jordb001,galaa001,jonea002,peree002,weisw001,maddg002,1
2000-04-03,4,20000403,0,Mon,SFN,1,FLO,1,4,6,...,castl001,gonza002,floyc001,wilsp002,lowem001,millk005,browb003,redmm001,ferna001,1


In [491]:
# How big is this games table?
print(games_ixd.shape)
#(41292, 96)

# When I loop through the game log table, I am going to update a column for "num_injuries" at any 
# game where we know a player on the disabled list played in THAT game and went on the DL right after.

# Create a column of zeros for num of injuries column to append to the games df that matches
# the games row length: 43720 rows long
games_ixd['injury']  = np.zeros((games_ixd.shape[0],), dtype=np.int)

(41292, 96)


In [492]:
games_ixd.head(2)

,real_index,date,num_game,day,v_team,v_team_game_num,h_team,h_team_game_num,v_team_score,h_team_score,...,h_pl_2_id,h_pl_3_id,h_pl_4_id,h_pl_5_id,h_pl_6_id,h_pl_7_id,h_pl_8_id,h_pl_9_id,intra_league,injury
dt_date,,,,,,,,,,,,,,,,,,,,,
2000-03-29,0,20000329,0,Wed,CHN,1,NYN,1,5,3,...,hamid001,alfoe001,piazm001,ventr001,belld001,zeilt001,ordor001,hampm001,1,0
2000-03-30,1,20000330,0,Thu,NYN,2,CHN,2,5,1,...,bufod001,sosas001,gracm001,rodrh001,andrs001,husoj001,giraj001,farnk001,1,0


### Subset games_ixd to just columns needed for finding DL list players: pl_id cols: real_index col and cols 71:95

In [495]:
# Create a small subset of the indexed games dataframe to only include the game date 
# and all the players' ids, cols 71:95
sm_games = pd.concat([games_ixd.iloc[:,0], games_ixd.iloc[:,71:95]], axis=1)
sm_games.shape

(41292, 25)

### Create a dictionary with real_index as the key and 0 as the values, to update += 1 every time a game row was the last game a DL player played in before his Disabled List placement date:

In [515]:
# This creates (and clears) the injury_update dictionary that will become the TARGET column:
injury_update_dict = games_ixd['injury']
injury_update_dict = injury_update_dict.reset_index()
injury_update_dict = injury_update_dict.drop('dt_date',axis=1).to_dict()
sum(injury_update_dict['injury'].values())

0

## Update injury_update_dict:
* Loop through all rows in the Disabled List data, and for every placement (dl_place), 
  * subset the games_ixd data to all games 10 days before that DL placement date to save search time. 
  * Loop through all games in the subset, and 
      * if the player_id from the placement is in the game, 
          * save that row index as the index to update. 
          
  * Increase the value of the injury_update_dict whose key is the index to update by + 1
 

### TESTING THE LOOP ON SUBSET:

In [402]:
# FOR PRACTICE RUNS THROUGH THE FOR LOOPS:
# Subset dataframes to practice on:
sm_games_subset = sm_games.iloc[:5000,:].copy()
sm_inj_subset = injury.iloc[:150,2:].copy()

injury_count = 0
for dl_place in sm_inj_subset.itertuples():
    print(dl_place)
    if dl_place[1] is not np.nan:
        pid = dl_place[1]
        search_end_day = dl_place[2]
        search_start_day = search_end_day-pd.offsets.Day(10)
        subset_games = sm_games_subset[search_start_day:search_end_day]
        for game in subset_games.itertuples():
            if pid in game:
                print('yes')
                print(game.real_index)
                game_index_to_update = game.real_index 
        injury_update_dict['injury'][game_index_to_update] += 1
        injury_count += 1

# Check how many injuries recorded        
print('injury count:', injury_count, 'dict count:',sum(injury_update_dict['injury'].values()))


Pandas(Index=0, player_id='thomj005', dt_date=Timestamp('2000-03-23 00:00:00'))
Pandas(Index=1, player_id='alvaw001', dt_date=Timestamp('2000-04-01 00:00:00'))
Pandas(Index=2, player_id='osuna002', dt_date=Timestamp('2000-04-01 00:00:00'))
Pandas(Index=3, player_id='hamid001', dt_date=Timestamp('2000-04-08 00:00:00'))
yes
0
yes
1
yes
30
yes
44
Pandas(Index=4, player_id='timlm001', dt_date=Timestamp('2000-04-08 00:00:00'))
Pandas(Index=5, player_id='hernc001', dt_date=Timestamp('2000-04-08 00:00:00'))
yes
5
yes
30
yes
44
Pandas(Index=6, player_id='wagnb001', dt_date=Timestamp('2000-06-21 00:00:00'))
yes
951
Pandas(Index=7, player_id='olivd001', dt_date=Timestamp('2000-06-21 00:00:00'))
yes
899
yes
971
Pandas(Index=8, player_id='cabra001', dt_date=Timestamp('2000-07-03 00:00:00'))
yes
1106
yes
1121
yes
1150
yes
1165
Pandas(Index=9, player_id=nan, dt_date=Timestamp('2000-07-07 00:00:00'))
Pandas(Index=10, player_id='ferno001', dt_date=Timestamp('2000-07-07 00:00:00'))
yes
1165
yes
1243
Pa

Pandas(Index=111, player_id=nan, dt_date=Timestamp('2001-04-21 00:00:00'))
Pandas(Index=112, player_id='osuna002', dt_date=Timestamp('2001-04-21 00:00:00'))
Pandas(Index=113, player_id='hillg001', dt_date=Timestamp('2001-04-22 00:00:00'))
yes
2554
yes
2564
yes
2594
yes
2608
yes
2631
yes
2649
yes
2660
Pandas(Index=114, player_id=nan, dt_date=Timestamp('2001-04-22 00:00:00'))
Pandas(Index=115, player_id=nan, dt_date=Timestamp('2001-04-24 00:00:00'))
Pandas(Index=116, player_id='walkk001', dt_date=Timestamp('2001-04-24 00:00:00'))
Pandas(Index=117, player_id=nan, dt_date=Timestamp('2001-04-25 00:00:00'))
Pandas(Index=118, player_id='stepg001', dt_date=Timestamp('2001-04-25 00:00:00'))
Pandas(Index=119, player_id='alvaw001', dt_date=Timestamp('2001-04-25 00:00:00'))
Pandas(Index=120, player_id='wittb001', dt_date=Timestamp('2001-04-25 00:00:00'))
Pandas(Index=121, player_id='leita001', dt_date=Timestamp('2001-04-25 00:00:00'))
yes
2664
Pandas(Index=122, player_id='cresf001', dt_date=Timest

# TRY IT OUT ON YOUR WHOLE DATASET:
* Make sure to clear out the injury_update_dict between iterations

In [516]:
def find_last_game(player_id, dl_date, games_df):
    """Function takes in a player_id and disabled list placement
    date and subsets the game logs to find the last date they played before
    the disabled list placement"""
    game_index_to_update = 999999
    search_end_day = dl_date
    search_start_day = search_end_day - pd.offsets.Day(200)
    subset_games = games_df[search_start_day:search_end_day]
    if subset_games.shape[0] > 0: # IF there are games 20 days before that day
        for game in subset_games.itertuples(): #loop through all games in 20 day period
            if player_id in game:
                game_index_to_update = game.real_index
            else:
                continue
    else: # If there were no games to check, return out of the function
        return game_index_to_update
    return game_index_to_update

In [517]:
# Use sm_games
sm_inj = injury.iloc[:,2:].copy()

injury_count = 0
for dl_place in sm_inj.itertuples():
    if dl_place[1] is not np.nan:
        index_to_update = find_last_game(dl_place[1],dl_place[2],sm_games)
        if index_to_update != 999999:
            injury_update_dict['injury'][index_to_update] += 1
            injury_count += 1
        else:
            continue
# Check how many injuries recorded        
print('injury count:', injury_count, 'dict count:',sum(injury_update_dict['injury'].values()))

injury count: 6396 dict count: 6396


### Add the values from the injury update dict back into the games_ixd

In [522]:
games_ixd['injury'] = games_ixd['real_index'].map(injury_update_dict['injury'])

In [524]:
games_ixd['injury'].value_counts()

0    36399
1     3611
2     1101
3      149
4       25
5        6
6        1
Name: injury, dtype: int64

In [526]:
games_prep = games_ixd.copy()

### One Hot Encode Categorical Columns before saving to CSV

In [527]:
games_prep.columns

Index(['real_index', 'date', 'num_game', 'day', 'v_team', 'v_team_game_num',
       'h_team', 'h_team_game_num', 'v_team_score', 'h_team_score',
       'game_length_outs', 'day_night', 'park_id', 'attendance',
       'time_game_min', 'v_at_bats', 'v_hits', 'v_doubles', 'v_triples',
       'v_homeruns', 'v_RBI', 'v_sac_hits', 'v_sac_files', 'v_hit_pitch',
       'v_walks', 'v_int_walks', 'v_strikeouts', 'v_stol_base',
       'v_caught_steal', 'v_grnd_dbl_plays', 'v_awd_fst_catch_intf',
       'v_left_on_base', 'v_pitchers', 'v_ind_earn_runs', 'v_team_earn_runs',
       'v_wild_pitch', 'v_balks', 'v_putouts', 'v_assists', 'v_errors',
       'v_pass_balls', 'v_dbl_plays', 'v_trp_plays', 'h_at_bats', 'h_hits',
       'h_doubles', 'h_triples', 'h_homeruns', 'h_RBI', 'h_sac_hits',
       'h_sac_files', 'h_hit_pitch', 'h_walks', 'h_int_walks', 'h_strikeouts',
       'h_stol_base', 'h_caught_steal', 'h_grnd_dbl_plays',
       'h_awd_fst_catch_intf', 'h_left_on_base', 'h_pitchers',
       'h_in

In [528]:
col = ['day', 'v_team', 'h_team', 'day_night', 'park_id']
games_w_dummies = pd.get_dummies(games_prep,columns=col, drop_first=True)

In [530]:
print(games_w_dummies.columns)
print(games_w_dummies.shape)

Index(['real_index', 'date', 'num_game', 'v_team_game_num', 'h_team_game_num',
       'v_team_score', 'h_team_score', 'game_length_outs', 'attendance',
       'time_game_min',
       ...
       'park_id_SFO03', 'park_id_SJU01', 'park_id_STL09', 'park_id_STL10',
       'park_id_STP01', 'park_id_SYD01', 'park_id_TOK01', 'park_id_TOR02',
       'park_id_WAS10', 'park_id_WAS11'],
      dtype='object', length=207)
(41292, 207)


## Save Full DataFrame to CSV for Further Investigation

In [531]:
games_w_dummies.to_csv("games_w_inj_dums.csv")

## Drop the 'real_index' and player id cols for the model:

In [537]:
cols_to_drop = [col for col in games_w_dummies.columns if ('id' in col) and ('park' not in col)]
cols_to_drop.append('real_index')
cols_to_keep = [item for item in games_w_dummies.columns if item not in cols_to_drop]
cols_to_keep

['date',
 'num_game',
 'v_team_game_num',
 'h_team_game_num',
 'v_team_score',
 'h_team_score',
 'game_length_outs',
 'attendance',
 'time_game_min',
 'v_at_bats',
 'v_hits',
 'v_doubles',
 'v_triples',
 'v_homeruns',
 'v_RBI',
 'v_sac_hits',
 'v_sac_files',
 'v_hit_pitch',
 'v_walks',
 'v_int_walks',
 'v_strikeouts',
 'v_stol_base',
 'v_caught_steal',
 'v_grnd_dbl_plays',
 'v_awd_fst_catch_intf',
 'v_left_on_base',
 'v_pitchers',
 'v_ind_earn_runs',
 'v_team_earn_runs',
 'v_wild_pitch',
 'v_balks',
 'v_putouts',
 'v_assists',
 'v_errors',
 'v_pass_balls',
 'v_dbl_plays',
 'v_trp_plays',
 'h_at_bats',
 'h_hits',
 'h_doubles',
 'h_triples',
 'h_homeruns',
 'h_RBI',
 'h_sac_hits',
 'h_sac_files',
 'h_hit_pitch',
 'h_walks',
 'h_int_walks',
 'h_strikeouts',
 'h_stol_base',
 'h_caught_steal',
 'h_grnd_dbl_plays',
 'h_awd_fst_catch_intf',
 'h_left_on_base',
 'h_pitchers',
 'h_ind_earn_runs',
 'h_team_earn_runs',
 'h_wild_pitch',
 'h_balks',
 'h_putouts',
 'h_assists',
 'h_errors',
 'h_pass_

In [555]:
games_X = games_w_dummies.loc[:,cols_to_keep]

In [556]:
games_X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41292 entries, 2000-03-29 to 2016-10-02
Columns: 182 entries, date to park_id_WAS11
dtypes: float64(1), int64(66), uint8(115)
memory usage: 26.0 MB


## Save the X DataFrame to CSV for Modeling

In [557]:
games_X.to_csv('games_features_for_model.csv')